In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt 
import seaborn as sns 

%matplotlib inline

In [2]:
# import pandas as pd
from random import shuffle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import pickle

# model
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score

# import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix

# Main functions

In [3]:
# remove some special characters
def remove_special_chars(sen, filter_chars):
    sen = sen.strip()
    sen = sen.lower()
    for each in sen:
        num_ascii = ord(each)
        # delete number, ".", "\", all chars in filter_chars
        if (num_ascii > 47 and num_ascii < 58) or num_ascii == 92 or num_ascii == 46 or (each in filter_chars):
            sen = sen.replace(each, "")
    return sen

In [9]:
# read file csv and convert it to pandasframe
def open_file(name):
    """
    
    """
    with open('{file_name}.csv'.format(file_name = "formatted_data"), encoding='Latin1') as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content] # mỗi

    data = []
    for num, each in enumerate(content):
        each = each.split(";")

        if "." in each[1]:
            sentences = each[1].split(".") 
            filter_chars = ['\t', '!', '"', '%', '&', '*', '+', ',', '-', '/', ':', '=', '?', '@', '[', ']', '§', 
                            '«', "”", "\\", ".", '»']
                    
            for number, sen in enumerate(sentences):
                """
                insert remove special characters

                """

                # filter no meaning words
                sen = remove_special_chars(sen, filter_chars)

                # make sure a sentence have len(sentence) > 0
                if len(sen)>0:
                    data.append([each[0], sen, each[2]])

        else:
            data.append(each)

    main_data = data[1:]
    main_data = shuffle(main_data)
    df = pd.DataFrame(main_data, columns = data[0])
    return df

In [10]:
df = open_file('formatted_data')

In [43]:
df.head()

,language,text,length_text
0,fi,komissio on kuitenkin sitã¤ mieltã¤ ettã¤ meid...,694523
1,de,die artikel,747690
2,en,we have to get this right,690268
3,fr,dans ce domaine il y a selon moi certaines zon...,756201
4,sk,schvã¡lenie zã¡pisnice z predchã¡dzajãºceho ro...,328185


In [11]:
df.shape

(78160, 3)

In [12]:

# get data in a row
def get_data(df = df, row = 60000):
    return  df.iloc[row][0], df.iloc[row][1]

label, text = get_data(row = 700)
print(label)
print(text)

fi
mietintã¶ todistaa kuitenkin ettã¤ alueelliset erot tyã¶markkinoilla ovat jopa syventyneet entisestã¤ã¤n


In [13]:
#label encoding

In [14]:
# vectorize sentences and split it in to train and test file
def vectorization(df, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(list(df["text"]), list(df["language"]), test_size=test_size, random_state=42)

    # vectorize sentence X
    count_vectorizer = CountVectorizer(analyzer='char')
    X_train_features = count_vectorizer.fit_transform(X_train)
    X_test_features = count_vectorizer.transform(X_test)

    # vectorize label Y
    label_encoder = preprocessing.LabelEncoder()
    y_train_features = label_encoder.fit_transform(y_train)
    y_test_features = label_encoder.transform(y_test)
    
    # getted features
    features = count_vectorizer.get_feature_names()
    
    # getted labels
    labels = list(label_encoder.classes_)
    
    return X_train_features, y_train_features, X_test_features, y_test_features, features, labels, count_vectorizer

X_train_features, y_train_features, X_test_features, y_test_features, features, labels, count_vectorizer = vectorization(df)

C:\Users\LIEN PHAM\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#word2vec, fasttext, bert

In [15]:
# number of the features
print("features: ", features)
print("\nLen features: ", len(features))

features:  [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x81', '\x82', '\x83', '\x84', '\x85', '\x86', '\x87', '\x88', '\x89', '\x8a', '\x8b', '\x8c', '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9a', '\x9b', '\x9c', '\x9d', '\x9e', '\x9f', '\xa0', '¡', '¢', '£', '¤', '¥', '¦', '¨', '©', 'ª', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '¼', '½', '¾', '¿', 'â', 'ã', 'ä', 'å', 'è', 'î', 'ï', 'ð', 'ñ']

Len features:  97


In [16]:
X_train_features.toarray()[0]

array([23, 12,  2,  9,  5, 11,  2,  1,  1, 15,  0,  0,  9,  4,  5, 11,  4,
        0, 10,  4, 13, 11,  0,  0,  0,  0,  2,  0,  0,  0,  3,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  3,  3,  0,  0,  0,  0,  0], dtype=int64)

In [17]:
# create and training model MultinomialNB
modelNB = MultinomialNB()
modelNB.fit(X_train_features, y_train_features)

# model accuracy, how often is the classifier correct?
print("Accuracy: ",f1_score(y_test_features, modelNB.predict(X_test_features), average='macro'))

Accuracy:  0.9230441160857645


In [19]:
# save the classifier
with open('modelNB.pkl', 'wb') as fid:
    pickle.dump(modelNB, fid) 
    
# load it again
with open('modelNB.pkl', 'rb') as fid:
    modelNB = pickle.load(fid)

In [20]:
# predict from saved model
modelNB.predict(X_test_features)

array([ 3, 16,  0, ..., 13,  2,  4], dtype=int64)

In [21]:
###Model Random Forest

In [22]:
# model Random Forest
modelRF=RandomForestClassifier(n_estimators=100)
modelRF.fit(X_train_features,y_train_features)

y_pred = modelRF.predict(X_test_features)

# model accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test_features, y_pred))

Accuracy: 0.9413382804503583


In [23]:
# save the classifier
with open('modelRF.pkl', 'wb') as fid:
    pickle.dump(modelRF, fid) 
    
# load it again
with open('modelRF.pkl', 'rb') as fid:
    modelRF = pickle.load(fid)

In [24]:
# predict from saved model
modelRF.predict(X_test_features)

array([ 2, 16,  0, ..., 13,  2,  4], dtype=int64)

In [25]:
###Optimize parametters for random forest model, accuracy can be better

In [26]:
# tuning model by grid search cv
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [50, 100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=RandomForestClassifier(), n_jobs = -1, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train_features, y_train_features)

# choose the best parametters 
print("Optimal paras: ", CV_rfc.best_params_)

Optimal paras:  {'max_features': 'log2', 'n_estimators': 200}


In [27]:
# Create random forest model with the optimal parametter
optimal_modelRF=RandomForestClassifier(n_estimators=300, max_features= 'log2')

#Train the model using the training sets 
optimal_modelRF.fit(X_train_features,y_train_features)

RandomForestClassifier(max_features='log2', n_estimators=300)

In [28]:
# find the accuracy
y_pred = optimal_modelRF.predict(X_test_features)

# model accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test_features, y_pred)) #Accuracy: 0.9437691914022518

Accuracy: 0.9437691914022518


In [30]:
# save the classifier
with open('optimal_modelRF.pkl', 'wb') as fid:
    pickle.dump(optimal_modelRF, fid) 
    
# load it again
with open('optimal_modelRF.pkl', 'rb') as fid:
    optimal_modelRF = pickle.load(fid)

In [ ]:
###Model support vector machine - SVM

In [31]:
model_SVM = SVC(kernel='linear')
model_SVM.fit(X_train_features, y_train_features)

y_pred = model_SVM.predict(X_test_features)

print(confusion_matrix(y_test_features,y_pred))
print(classification_report(y_test_features,y_pred))

[[ 656    0    0    4    0    5    2    0    0    1    0    1    0    0
     0    0    0    5    0    0    0]
 [  13  470    1    2    1    2    2    2    0    0    4    0    1    0
     0    0    1    1   17    2    0]
 [   1    0 1000    4    0    2    2    0    0    2    0    1    1    3
    12    0    0    0    0    0   13]
 [   2    0   11  882    0    5    0    4    0    0    0    1    1    5
    15    2    0    1    0    1    4]
 [   2    0    1    0  587    0    1    0    0    0    0    0    0    0
     0    0    0    0    0    0    0]
 [   1    0    5   10    0  876    9    0    0    2    0    3    6    4
     8    0    0    0    0    0    0]
 [   0    0    7    2    0    7  870    0    0   15    2    3    1    2
     1    0   17    1    1    0    0]
 [   2    0    4    5    0    3    7  472    9    1    0    0    2    4
     1    0    1    2    0    2    3]
 [   0    0    2    1    0    0    1    7  915    1    0    0    0    0
     0    0    0    0    0    0    1]
 [   1    

In [ ]:
###Find the best model

In [32]:
X, Y = list(df["text"]), list(df["language"])

# vectorize sentence X
vectorizer = CountVectorizer(analyzer='char')
X_features = count_vectorizer.fit_transform(X)

# vectorize label Y
label_encoder = preprocessing.LabelEncoder()
Y_features = label_encoder.fit_transform(Y)

In [33]:
models = [
    RandomForestClassifier(n_estimators=200),
    SVC(),
    GaussianNB(),
    LogisticRegression(),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models))) # no need
entries = []

In [34]:
X_features.toarray()

array([[ 9,  0,  0, ...,  0,  0,  0],
       [ 2,  1,  0, ...,  0,  0,  0],
       [ 5,  1,  0, ...,  0,  0,  0],
       ...,
       [10,  0,  0, ...,  0, 59, 16],
       [21,  0,  0, ..., 41,  0,  0],
       [10,  9,  1, ...,  0,  0,  0]], dtype=int64)

In [35]:
Y_features

array([8, 3, 5, ..., 0, 4, 6], dtype=int64)

In [36]:
len(X_features.toarray()[0])

97

In [37]:
len(X_features.toarray())

78160

In [38]:
len(Y_features)

78160

In [40]:
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, 
                                 X_features.toarray(), 
                                 Y_features, 
                                 scoring='accuracy', 
                                 cv=CV, 
                                 n_jobs=-1                                
                                 )
    print(accuracies)    
    entries.append([model_name, sum(accuracies)/len(accuracies)])

cv_df = pd.DataFrame(entries, columns=['model_name', 'accuracy'])



[0.94172211 0.94121034 0.94037871 0.94089048 0.94082651]
[0.90084442 0.89495906 0.90007677 0.89905322 0.89636643]
[0.79881013 0.80188076 0.80040942 0.80098516 0.81275589]
[0.93519703 0.93046315 0.93423746 0.93353378 0.93142272]


In [41]:
cv_df

,model_name,accuracy
0,RandomForestClassifier,0.941006
1,SVC,0.898260
2,GaussianNB,0.802968
3,LogisticRegression,0.932971


In [ ]:
[0.94079284 0.9434638  0.9470957  0.9484226  0.937408  ]
[0.91432225 0.91385265 0.91613357 0.92116209 0.913344  ]
[0.81675192 0.82693784 0.82970829 0.81928713 0.81888   ]
[0.93842711 0.93789972 0.9392912  0.94176745 0.930752  ]

In [ ]:
model_name	accuracy
0	RandomForestClassifier	0.941006
1	SVC	0.898260
2	GaussianNB	0.802968
3	LogisticRegression	0.932971

In [ ]:
#Comments: RF has highest accuracy rate